## Build an MTH5 and Operate the Aurora Pipeline

Outlines the process of making an MTH5 file, generating a processing config, and running the aurora processor

In [1]:
# Required imports for theh program. 
from pathlib import Path
import sys
import pandas as pd
from mth5.clients.make_mth5 import MakeMTH5
from mth5 import mth5, timeseries
from mt_metadata.utils.mttime import get_now_utc, MTime
from aurora.config.config_creator import ConfigCreator
from aurora.pipelines.process_mth5 import process_mth5_run

2021-11-19 18:55:35,163 [line 135] mth5.setup_logger - INFO: Logging file can be found /Users/tronan/Desktop/Projects/mth5/logs/mth5_debug.log


Build an MTH5 file from information extracted by IRIS

In [2]:
# Set path so MTH5 file builds to current working directory. 
default_path = Path().cwd()

In [3]:
# Initialize the Make MTH5 code. 
m = MakeMTH5(mth5_version='0.1.0')
m.client = "IRIS"

In [4]:
# Generate data frame of FDSN Network, Station, Location, Channel, Startime, Endtime codes of interest

ZUCAS04LQ1 = ['ZU', 'CAS04', '', 'LQE', '2020-06-02T19:00:00', '2020-07-13T19:00:00']
ZUCAS04LQ2 = ['ZU', 'CAS04', '', 'LQN', '2020-06-02T19:00:00', '2020-07-13T19:00:00']
ZUCAS04BF1 = ['ZU', 'CAS04', '', 'LFE', '2020-06-02T19:00:00', '2020-07-13T19:00:00']
ZUCAS04BF2 = ['ZU', 'CAS04', '', 'LFN', '2020-06-02T19:00:00', '2020-07-13T19:00:00']
ZUCAS04BF3 = ['ZU', 'CAS04', '', 'LFZ', '2020-06-02T19:00:00', '2020-07-13T19:00:00']

request_list = [ZUCAS04LQ1, ZUCAS04LQ2, ZUCAS04BF1, ZUCAS04BF2, ZUCAS04BF3]

# Turn list into dataframe
request_df =  pd.DataFrame(request_list, columns=m.column_names)\

In [5]:
# Inspect the dataframe
print(request_df)

  network station location channel                start                  end
0      ZU   CAS04              LQE  2020-06-02T19:00:00  2020-07-13T19:00:00
1      ZU   CAS04              LQN  2020-06-02T19:00:00  2020-07-13T19:00:00
2      ZU   CAS04              LFE  2020-06-02T19:00:00  2020-07-13T19:00:00
3      ZU   CAS04              LFN  2020-06-02T19:00:00  2020-07-13T19:00:00
4      ZU   CAS04              LFZ  2020-06-02T19:00:00  2020-07-13T19:00:00


In [6]:
# Request the inventory information from IRIS
inventory = m.get_inventory_from_df(request_df, data=False)

In [7]:
# Inspect the inventory
inventory

(Inventory created at 2021-11-20T02:55:41.899160Z
	Created by: ObsPy 1.2.2
		    https://www.obspy.org
	Sending institution: MTH5
	Contains:
		Networks (1):
			ZU
		Stations (1):
			ZU.CAS04 (Corral Hollow, CA, USA)
		Channels (5):
			ZU.CAS04..LFZ, ZU.CAS04..LFN, ZU.CAS04..LFE, ZU.CAS04..LQN, 
			ZU.CAS04..LQE,
 0 Trace(s) in Stream:
)

Builds an MTH5 file from the user defined database. 
Note: Intact keeps the MTH5 open after it is done building


In [8]:
mth5_object = m.make_mth5_from_fdsnclient(request_df, interact=True)

2021-11-19 18:55:45,864 [line 594] mth5.mth5.MTH5.open_mth5 - WARNING: ZU_CAS04.h5 will be overwritten in 'w' mode
2021-11-19 18:55:46,244 [line 653] mth5.mth5.MTH5._initialize_file - INFO: Initialized MTH5 file /Users/tronan/Desktop/Projects/aurora/docs/notebooks/ZU_CAS04.h5 in mode w
2021-11-19 18:55:56,255 [line 127] mt_metadata.base.metadata.station.add_run - WARNING: Run a is being overwritten with current information


2021-11-19T18:55:56 [line 127] obspy_stages.create_filter_from_stage - INFO: Converting PoleZerosResponseStage mV/km to V/m to a CoefficientFilter
2021-11-19T18:55:56 [line 127] obspy_stages.create_filter_from_stage - INFO: Converting PoleZerosResponseStage V/m to V to a CoefficientFilter
2021-11-19T18:55:56 [line 127] obspy_stages.create_filter_from_stage - INFO: Converting PoleZerosResponseStage mV/km to V/m to a CoefficientFilter
2021-11-19T18:55:56 [line 127] obspy_stages.create_filter_from_stage - INFO: Converting PoleZerosResponseStage V/m to V to a CoefficientFilter


2021-11-19 18:55:58,165 [line 709] mth5.groups.base.Station.add_run - INFO: run a already exists, returning existing group.
2021-11-19 18:55:58,469 [line 224] mth5.timeseries.run_ts.RunTS.validate_metadata - WARNING: start time of dataset 2020-06-02T19:00:00+00:00 does not match metadata start 2020-06-02T18:41:43+00:00 updating metatdata value to 2020-06-02T19:00:00+00:00
2021-11-19 18:56:04,843 [line 709] mth5.groups.base.Station.add_run - INFO: run b already exists, returning existing group.
2021-11-19 18:56:13,102 [line 709] mth5.groups.base.Station.add_run - INFO: run c already exists, returning existing group.
2021-11-19 18:56:23,571 [line 709] mth5.groups.base.Station.add_run - INFO: run d already exists, returning existing group.
2021-11-19 18:56:25,105 [line 235] mth5.timeseries.run_ts.RunTS.validate_metadata - WARNING: end time of dataset 2020-07-13T19:00:00+00:00 does not match metadata end 2020-07-13T21:46:12+00:00 updating metatdata value to 2020-07-13T19:00:00+00:00


In [9]:
# mth5_object.open_mth5(h5_path, 'w')
# h5_path = str(default_path)+'/ZU_CAS04.h5'
#mth5_object.close_mth5()

Extract information from the open MTH5 Object 

In [10]:
mth5_object

/:
    |- Group: Survey
    ----------------
        |- Group: Filters
        -----------------
            |- Group: coefficient
            ---------------------
                |- Group: mv per km to v per m
                ------------------------------
                |- Group: v per m to v
                ----------------------
                |- Group: v to counts (electric)
                --------------------------------
                |- Group: v to counts (magnetic)
                --------------------------------
            |- Group: fap
            -------------
            |- Group: fir
            -------------
            |- Group: time_delay
            --------------------
                |- Group: electric time offset
                ------------------------------
                |- Group: hx time offset
                ------------------------
                |- Group: hy time offset
                ------------------------
                |- Group: hz time offse

In [11]:
# Collect information from the MTh5 Object and use it in the config files. 
mth5_filename = mth5_object.filename
version = mth5_object.file_version

In [12]:
# Edit and update the MTH5 metadata 
s = mth5_object.get_station("CAS04")
s.metadata.location.declination.model = 'IGRF'
s.write_metadata()

In [13]:
# Get the available stations and runs from the MTH5 object
ch_summary = mth5_object.channel_summary

In [14]:
ch_summary

,station,run,latitude,longitude,elevation,component,start,end,n_samples,sample_rate,measurement_type,azimuth,tilt,units,hdf5_reference
0,CAS04,a,37.633351,-121.468382,329.3875,ex,2020-06-02 19:00:00,2020-06-02 22:07:46,11267,1.0,electric,13.2,0.0,digital counts,<HDF5 object reference>
1,CAS04,a,37.633351,-121.468382,329.3875,ey,2020-06-02 19:00:00,2020-06-02 22:07:46,11267,1.0,electric,103.2,0.0,digital counts,<HDF5 object reference>
2,CAS04,a,37.633351,-121.468382,329.3875,hx,2020-06-02 19:00:00,2020-06-02 22:07:46,11267,1.0,magnetic,13.2,0.0,digital counts,<HDF5 object reference>
3,CAS04,a,37.633351,-121.468382,329.3875,hy,2020-06-02 19:00:00,2020-06-02 22:07:46,11267,1.0,magnetic,103.2,0.0,digital counts,<HDF5 object reference>
4,CAS04,a,37.633351,-121.468382,329.3875,hz,2020-06-02 19:00:00,2020-06-02 22:07:46,11267,1.0,magnetic,0.0,0.0,digital counts,<HDF5 object reference>
5,CAS04,b,37.633351,-121.468382,329.3875,ex,2020-06-02 22:24:55,2020-06-12 17:52:23,847649,1.0,electric,13.2,0.0,digital counts,<HDF5 object reference>
6,CAS04,b,37.633351,-121.468382,329.3875,ey,2020-06-02 22:24:55,2020-06-12 17:52:23,847649,1.0,electric,103.2,0.0,digital counts,<HDF5 object reference>
7,CAS04,b,37.633351,-121.468382,329.3875,hx,2020-06-02 22:24:55,2020-06-12 17:52:23,847649,1.0,magnetic,13.2,0.0,digital counts,<HDF5 object reference>
8,CAS04,b,37.633351,-121.468382,329.3875,hy,2020-06-02 22:24:55,2020-06-12 17:52:23,847649,1.0,magnetic,103.2,0.0,digital counts,<HDF5 object reference>
9,CAS04,b,37.633351,-121.468382,329.3875,hz,2020-06-02 22:24:55,2020-06-12 17:52:23,847649,1.0,magnetic,0.0,0.0,digital counts,<HDF5 object reference>


In [15]:
available_runs = ch_summary.run.unique()
sr = ch_summary.sample_rate.unique()
if len(sr) != 1:
    print('Only one sample rate per run is available')
available_stations = ch_summary.station.unique()

In [16]:
sr[0]

1.0

In [17]:
available_stations[0]

'CAS04'

In [18]:
mth5_object

/:
    |- Group: Survey
    ----------------
        |- Group: Filters
        -----------------
            |- Group: coefficient
            ---------------------
                |- Group: mv per km to v per m
                ------------------------------
                |- Group: v per m to v
                ----------------------
                |- Group: v to counts (electric)
                --------------------------------
                |- Group: v to counts (magnetic)
                --------------------------------
            |- Group: fap
            -------------
            |- Group: fir
            -------------
            |- Group: time_delay
            --------------------
                |- Group: electric time offset
                ------------------------------
                |- Group: hx time offset
                ------------------------
                |- Group: hy time offset
                ------------------------
                |- Group: hz time offse

Generate an Aurora Configuration file using MTH5 as an input

In [19]:
station_id = available_stations[0]
run_id = available_runs[0]
sample_rate = sr[0]
config_maker = ConfigCreator()
config_path = config_maker.create_run_config(station_id, run_id, mth5_filename, sample_rate)


In [20]:
config_path

PosixPath('config/CAS04-a_run_config.json')

Run the Aurora Pipeline using the input MTh5 and Confiugration File

In [21]:
show_plot='True'

In [ ]:
tf_cls = process_mth5_run(
        config_path,
        run_id,
        mth5_path=mth5_filename,
        units="MT",
        show_plot=False,
        z_file_path=None,
        return_collection=False,
    )

In [23]:

type(tf_cls)

mt_metadata.transfer_functions.core.TF

Write the transfer functions generated by the Aurora pipeline

In [25]:
 tf_cls.write_tf_file(fn="emtfxml_test.xml", file_type="emtfxml")

{
    "e_m_t_f_x_m_l": {
        "description": "USMTArray South Magnetotelluric Time Series (USMTArray CONUS South-USGS)",
        "product_id": "CAS04",
        "sub_type": "MT_TF",
        "tags": "impedance, tipper"
    }
}

In [ ]:
tf_cls.write_tf_file(fn="emtfxml_test.xml", file_type="edi")

In [ ]:
 tf_cls.write_tf_file(fn="emtfxml_test.xml", file_type="zmm")